In [0]:
!git clone https://github.com/google-research/bert.git

Cloning into 'bert'...
remote: Enumerating objects: 4, done.
remote: Counting objects: 100% (4/4), done.
remote: Compressing objects: 100% (4/4), done.
remote: Total 340 (delta 0), reused 0 (delta 0), pack-reused 336
Receiving objects: 100% (340/340), 315.29 KiB | 4.09 MiB/s, done.
Resolving deltas: 100% (183/183), done.


### **Confirming that BERT repo is cloned properly.**


> "ls -l" is used for long listing, if BERT repo is cloned properly we can see the BERT folder in current directory.



In [0]:
ls -l

total 8
drwxr-xr-x 3 root root 4096 Mar 11 14:38 bert/
drwxr-xr-x 1 root root 4096 Mar  3 18:11 sample_data/


In [0]:
cd bert

/content/bert


### **BERT repository files**


> use ls -l to check the content inside BERT folder, you can see all files related to BERT.



In [0]:
ls -l

total 396
-rw-r--r-- 1 root root  1323 Mar 11 14:38 CONTRIBUTING.md
-rw-r--r-- 1 root root 16475 Mar 11 14:38 create_pretraining_data.py
-rw-r--r-- 1 root root 13898 Mar 11 14:38 extract_features.py
-rw-r--r-- 1 root root   616 Mar 11 14:38 __init__.py
-rw-r--r-- 1 root root 11358 Mar 11 14:38 LICENSE
-rw-r--r-- 1 root root 37922 Mar 11 14:38 modeling.py
-rw-r--r-- 1 root root  9191 Mar 11 14:38 modeling_test.py
-rw-r--r-- 1 root root 11242 Mar 11 14:38 multilingual.md
-rw-r--r-- 1 root root  6258 Mar 11 14:38 optimization.py
-rw-r--r-- 1 root root  1721 Mar 11 14:38 optimization_test.py
-rw-r--r-- 1 root root 66488 Mar 11 14:38 predicting_movie_reviews_with_bert_on_tf_hub.ipynb
-rw-r--r-- 1 root root 45390 Mar 11 14:38 README.md
-rw-r--r-- 1 root root   110 Mar 11 14:38 requirements.txt
-rw-r--r-- 1 root root 34783 Mar 11 14:38 run_classifier.py
-rw-r--r-- 1 root root 11426 Mar 11 14:38 run_classifier_with_tfhub.py
-rw-r--r-- 1 root root 18667 Mar 11 14:38 run_pretraining.py
-rw-r--r-

In [0]:
!wget https://storage.googleapis.com/bert_models/2018_10_18/uncased_L-24_H-1024_A-16.zip

--2020-03-12 16:03:31--  https://storage.googleapis.com/bert_models/2018_10_18/uncased_L-24_H-1024_A-16.zip
Resolving storage.googleapis.com (storage.googleapis.com)... 64.233.183.128, 2607:f8b0:4001:c05::80
Connecting to storage.googleapis.com (storage.googleapis.com)|64.233.183.128|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1247797031 (1.2G) [application/zip]
Saving to: ‘uncased_L-24_H-1024_A-16.zip’

uncased_L-24_H-1024 100%[===================>]   1.16G   209MB/s    in 6.1s    

2020-03-12 16:03:38 (195 MB/s) - ‘uncased_L-24_H-1024_A-16.zip’ saved [1247797031/1247797031]



In [0]:
# Unzip the pretrained model
!unzip uncased_L-24_H-1024_A-16.zip

Archive:  uncased_L-24_H-1024_A-16.zip
   creating: uncased_L-24_H-1024_A-16/
  inflating: uncased_L-24_H-1024_A-16/bert_model.ckpt.meta  
  inflating: uncased_L-24_H-1024_A-16/bert_model.ckpt.data-00000-of-00001  
  inflating: uncased_L-24_H-1024_A-16/vocab.txt  
  inflating: uncased_L-24_H-1024_A-16/bert_model.ckpt.index  
  inflating: uncased_L-24_H-1024_A-16/bert_config.json  


**Download the SQUAD 2.0 Dataset**

In [0]:
#Download the SQUAD train and dev dataset
!wget https://rajpurkar.github.io/SQuAD-explorer/dataset/train-v2.0.json
!wget https://rajpurkar.github.io/SQuAD-explorer/dataset/dev-v2.0.json

--2020-03-11 14:40:45--  https://rajpurkar.github.io/SQuAD-explorer/dataset/train-v2.0.json
Resolving rajpurkar.github.io (rajpurkar.github.io)... 185.199.109.153, 185.199.108.153, 185.199.110.153, ...
Connecting to rajpurkar.github.io (rajpurkar.github.io)|185.199.109.153|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 42123633 (40M) [application/json]
Saving to: ‘train-v2.0.json’

train-v2.0.json     100%[===================>]  40.17M   175MB/s    in 0.2s    

2020-03-11 14:40:45 (175 MB/s) - ‘train-v2.0.json’ saved [42123633/42123633]

--2020-03-11 14:40:48--  https://rajpurkar.github.io/SQuAD-explorer/dataset/dev-v2.0.json
Resolving rajpurkar.github.io (rajpurkar.github.io)... 185.199.109.153, 185.199.108.153, 185.199.110.153, ...
Connecting to rajpurkar.github.io (rajpurkar.github.io)|185.199.109.153|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 4370528 (4.2M) [application/json]
Saving to: ‘dev-v2.0.json’

dev-v2.0.json       1

### **Setting up the TPU environment**
*   Verifying the connection to a TPU device
*   Obtaining TPU address
*   Performing Google Authentication to access the bucket
*   Uploading credentials to TPU to access the GCS bucket

In [0]:
import datetime
import json
import os
import pprint
import random
import string
import sys
import tensorflow as tf

assert 'COLAB_TPU_ADDR' in os.environ, 'ERROR: Not connected to a TPU runtime; please see the first cell in this notebook for instructions!'
TPU_ADDRESS = 'grpc://' + os.environ['COLAB_TPU_ADDR']
print('TPU address is => ', TPU_ADDRESS)

from google.colab import auth
auth.authenticate_user()



TPU address is =>  grpc://10.93.217.106:8470


In [0]:
with tf.Session(TPU_ADDRESS) as session:
  print('TPU devices:')
  pprint.pprint(session.list_devices())

  # Upload credentials to TPU.
  with open('/content/adc.json', 'r') as f:
    auth_info = json.load(f)
  tf.contrib.cloud.configure_gcs(session, credentials=auth_info)
  # Now credentials are set for all future sessions on this TPU.

NameError: ignored

### **Create output directory** 


> Need to create a output directory at GCS (Google Cloud Storage) bucket, where you will get your fine_tuned model after training completion. For that you need to provide your BUCKET name and OUPUT DIRECTORY name.

> Also need to move Pre-trained Model at GCS (Google Cloud Storage) bucket, as Local File System is not Supported on TPU. If you don't move your pretrained model to TPU you may face an error. 




In [0]:
  BUCKET = 'bert-question-answer' #@param {type:"string"}
  assert BUCKET, '*** Must specify an existing GCS bucket name ***'
  output_dir_name = 'bert_output_new' #@param {type:"string"}
  BUCKET_NAME = 'gs://{}'.format(BUCKET)
  OUTPUT_DIR = 'gs://{}/{}'.format(BUCKET,output_dir_name)
  tf.gfile.MakeDirs(OUTPUT_DIR)
  print('***** Model output directory: {} *****'.format(OUTPUT_DIR))

NameError: ignored

### **Move Pretrained Model to GCS Bucket** 


> Moving the Pre-trained Model at GCS (Google Cloud Storage) bucket, as Local File System is not Supported on TPU. gsutil mv command is used.



In [0]:
!gsutil mv /content/bert/uncased_L-24_H-1024_A-16 $BUCKET_NAME

Copying file:///content/bert/uncased_L-24_H-1024_A-16/bert_model.ckpt.index [Content-Type=application/octet-stream]...
Removing file:///content/bert/uncased_L-24_H-1024_A-16/bert_model.ckpt.index...
Copying file:///content/bert/uncased_L-24_H-1024_A-16/bert_model.ckpt.data-00000-of-00001 [Content-Type=application/octet-stream]...
==> NOTE: You are uploading one or more large file(s), which would run
significantly faster if you enable parallel composite uploads. This
feature can be enabled by editing the
"parallel_composite_upload_threshold" value in your .boto
configuration file. However, note that if you do this large files will
be uploaded as `composite objects
<https://cloud.google.com/storage/docs/composite-objects>`_,which
means that any user who downloads such objects will need to have a
compiled crcmod installed (see "gsutil help crcmod"). This is because
without a compiled crcmod, computing checksums on composite objects is
so slow that gsutil disables downloads of composite ob

### **Training**






In [0]:
!python run_squad.py \
  --vocab_file=$BUCKET_NAME/uncased_L-24_H-1024_A-16/vocab.txt \
  --bert_config_file=$BUCKET_NAME/uncased_L-24_H-1024_A-16/bert_config.json \
  --init_checkpoint=$BUCKET_NAME/uncased_L-24_H-1024_A-16/bert_model.ckpt \
  --do_train=True \
  --train_file=train-v2.0.json \
  --do_predict=True \
  --predict_file=dev-v2.0.json \
  --train_batch_size=24 \
  --learning_rate=3e-5 \
  --num_train_epochs=2.0 \
  --use_tpu=True \
  --tpu_name=grpc://10.115.214.210:8470 \
  --max_seq_length=384 \
  --doc_stride=128 \
  --version_2_with_negative=True \
  --output_dir=$OUTPUT_DIR

Streaming output truncated to the last 5000 lines.
I0311 15:01:47.921363 139666637977472 tpu_estimator.py:604] Dequeue next (1) batch(es) of data from outfeed.
INFO:tensorflow:Enqueue next (1) batch(es) of data to infeed.
I0311 15:01:47.940225 139666637977472 tpu_estimator.py:600] Enqueue next (1) batch(es) of data to infeed.
INFO:tensorflow:Dequeue next (1) batch(es) of data from outfeed.
I0311 15:01:47.940420 139666637977472 tpu_estimator.py:604] Dequeue next (1) batch(es) of data from outfeed.
INFO:tensorflow:Enqueue next (1) batch(es) of data to infeed.
I0311 15:01:47.959145 139666637977472 tpu_estimator.py:600] Enqueue next (1) batch(es) of data to infeed.
INFO:tensorflow:Dequeue next (1) batch(es) of data from outfeed.
I0311 15:01:47.959338 139666637977472 tpu_estimator.py:604] Dequeue next (1) batch(es) of data from outfeed.
INFO:tensorflow:Enqueue next (1) batch(es) of data to infeed.
I0311 15:01:47.991976 139666637977472 tpu_estimator.py:600] Enqueue next (1) batch(es) of data

In [0]:
!python /content/evaluate-v2.0.py /content/bert/dev-v2.0.json /content/bert_output_predictions.json


{
  "exact": 75.98753474269351,
  "f1": 79.25590666413233,
  "total": 11873,
  "HasAns_exact": 76.82186234817814,
  "HasAns_f1": 83.36797905250371,
  "HasAns_total": 5928,
  "NoAns_exact": 75.1555929352397,
  "NoAns_f1": 75.1555929352397,
  "NoAns_total": 5945
}


In [0]:
!git clone https://github.com/google-research/bert.git


Cloning into 'bert'...
remote: Enumerating objects: 340, done.
remote: Total 340 (delta 0), reused 0 (delta 0), pack-reused 340
Receiving objects: 100% (340/340), 300.28 KiB | 4.00 MiB/s, done.
Resolving deltas: 100% (185/185), done.
